# Clean the Texas OIS dataset for analysis -- officers shot data

### Latest run covers incidents from 2015-10-06 to 2018-03-25

* Inputs:
   * `OIS.xlsx` (currently local -- TODO add to data.world)
   * `texas_law_enforcement_agencies_and_counties.csv` (dtw - used to add county information)
* Output: `shot_officers.csv`

##### Author: Everett Wetchler (everett.wetchler@gmail.com)

## 1. Setup and read data

In [1]:
DTW_PROJECT_KEY = 'tji/auxiliary-datasets'
AGENCY_COUNTY_DATAFRAME_NAME = 'agencies_and_counties'
# TODO(@wetchler): move OIS data to data.world
RAW_FILENAME = '../data/raw/OIS.xlsx'
CLEANED_FILENAME = '../data/clean/shot_officers.csv'

In [2]:
import datadotworld as dw
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

%load_ext watermark
%watermark -a "Everett Wetchler" -d -t -z -r -g -w -p datadotworld,numpy,pandas

Everett Wetchler 2018-05-09 20:03:53 CDT

datadotworld 1.6.0
numpy 1.14.3
pandas 0.22.0
Git hash: 3686dd16f13dc2713dba7919fb5378e68ab1cf4c
Git repo: git@github.com:texas-justice-initiative/data-processing.git
watermark 1.6.0


In [3]:
from lib.standardize_police_agency_names import standardize_agency_name
from lib.cleaning_tools import *

In [4]:
dtw_datasets = dw.load_dataset(DTW_PROJECT_KEY, force_update=True)
agencies = dtw_datasets.dataframes[AGENCY_COUNTY_DATAFRAME_NAME]
agencies.head()

,agency,county
0,ANDERSON CO CONST PCT 1,ANDERSON
1,ANDERSON CO CONST PCT 2,ANDERSON
2,ANDERSON CO CONST PCT 3,ANDERSON
3,ANDERSON CO CONST PCT 4,ANDERSON
4,ANDERSON CO DIST ATTY OFFICE,ANDERSON


In [5]:
SHEET_NAME = 'OfficersShot'
shootings = pd.read_excel(RAW_FILENAME, sheetname=SHEET_NAME)
print("OIS officers-shot incidents from %s to %s" % (
    shootings['Date of Incident'].min().strftime('%Y-%m-%d'),
    shootings['Date of Incident'].max().strftime('%Y-%m-%d')))
shootings.head()

/Users/everettwetchler/.virtualenvs/jupyter-data-processing/lib/python3.6/site-packages/pandas/util/_decorators.py:118: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


OIS officers-shot incidents from 2015-10-06 to 2018-03-25


,No.,Number of Reports Filed,Date OAG Received,Date of Report 1,Name of Agency 1,City of Agency 1,Zip code of Agency 1,Date of Incident,Name of Person 1 Filling out Form,Email Address of Person 1 Filling out Form,Date of Report 2,Name of Agency 2,City of Agency 2,Zip code of Agency 2,Name of Person 2 Filling out form,Email Address of Person 2 Filling out Form,Officer First Name,Officer Last Name,Injured or Deceased's Gender,Injured or Deceased's Age,Injured or Deceased's Race/Ethnicity,Street Address of Incident,City of Incident,County of Incident,Zip Code of Incident,Latitude of Incident,Longitude of Incident,Incident Resulted In 1,Incident Resulted In 2,Non-Peace Officer's Gender 1,Non-Peace Officer's Age 1,Non-Peace Officer's Race/Ethnicity 1,Non-Officer First Name 1,Non-Officer Last Name 1,Non-Peace Officer's Gender 2,Non-Peace Officer's Age 2,Non-Peace Officer's Race/Ethnicity 2,Non-Officer First Name 2,Non-Officer Last Name 2,Non-Peace Officer's Gender 3,Non-Peace Officer's Age 3,Non-Peace Officer's Race/Ethnicity 3,Non-Officer First Name 3,Non-Officer Last Name 3,Media,Column1,Column2
0,1,1,NaT,10/26/2015,Seguin Police Department,Seguin,78155,2015-10-06,MC Meyers,mcmeyers@seguintexas.gov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Female,54,Anglo or White,815 Lamar Drive,Seguin,Guadalupe,78155,29.576609,-97.947667,Injury of Peace Officer,NaN,Female,54,Anglo or White,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,NaT,11/3/2015,Bexar County Sheriff's Office,San Antonio,78207,2015-10-30,Det. Frank Stubbs #4048,sstubbs@bexar.org,NaN,NaN,NaN,NaN,NaN,NaN,Joseph,Canales,Male,42,Hispanic or Latino,8000 Midcrown,San Antonio,Bexar,78218,29.507478,-98.380029,Injury of Peace Officer,NaN,Male,36,Black or African American,Sherman,Robinson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MySA,News4SanAntonio,NaN
2,3,1,2017-02-21,2/21/2017,Marlin Police Department,Marlin,76661,2015-11-01,Damien Eaglin,chiefeaglin@marlintx.net,NaN,NaN,NaN,NaN,NaN,NaN,Darrell,Allen,Male,42,Black or African American,301 W. Avenue B,Temple,Bell County,76501,31.095688,-97.344775,Death of Peace Officer,NaN,Male,Not Available,Not Available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Waco Tribune-Herald,NaN,NaN
3,4,1,NaT,12/29/2015,Odessa Police Department,Odessa,79761,2015-12-23,Sgt. David Lara,dlara@odessa-tx.gov,NaN,NaN,NaN,NaN,NaN,NaN,Pete,Gonzales,Male,42,Hispanic or Latino,1025 Custer Ave.,Odessa,Ector,79761,31.865143,-102.335850,Injury of Peace Officer,NaN,Male,27,Hispanic or Latino,Roy Daniel,Garza,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CBS 7,Big Country,NaN
4,5,1,NaT,12/29/2015,Odessa Police Department,Odessa,79761,2015-12-23,Sgt. David Lara,dlara@odessa-tx.gov,NaN,NaN,NaN,NaN,NaN,NaN,Cory,Wester,Male,38,Anglo or White,1025 Custer Ave.,Odessa,Ector,79761,31.865143,-102.335850,Injury of Peace Officer,NaN,Male,27,Hispanic or Latino,Roy Daniel,Garza,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CBS 7,Big Country,NaN


## 2. Begin cleaning

### Remove whitespace from column names

In [6]:
shootings.columns = [c.strip().lower() for c in shootings.columns]

### Uppercase and strip all string cells

In [7]:
for col, dt in shootings.dtypes.items():
    if dt == 'object':
        print(col, end=', ')
        shootings[col] = shootings[col].apply(lambda s: s if type(s) != str else s.strip().upper())

date of report 1, name of agency 1, city of agency 1, zip code of agency 1, name of person 1 filling out form, email address of person 1 filling out form, date of report 2, name of agency 2, city of agency 2, name of person 2 filling out form, email address of person 2 filling out form, officer first name, officer last name, injured or deceased's gender, injured or deceased's race/ethnicity, street address of incident, city of incident, county of incident, incident resulted in 1, incident resulted in 2, non-peace officer's gender 1, non-peace officer's age 1, non-peace officer's race/ethnicity 1, non-officer first name 1, non-officer last name 1, non-peace officer's gender 2, non-peace officer's race/ethnicity 2, non-officer first name 2, non-officer last name 2, non-peace officer's gender 3, non-peace officer's race/ethnicity 3, media, column1, column2, 

### Drop unnecessary columns

In [8]:
shootings.drop(['no.', 'number of reports filed'], axis=1, inplace=True)

### Make column names more machine-friendly

In [9]:
col_renames = {
    "date oag received": "date_ag_received",
    "date of incident": "date_incident",
    "officer first name": "officer_name_first",
    "officer last name": "officer_name_last",
    "injured or deceased's gender": "officer_gender",
    "injured or deceased's age": "officer_age",
    "injured or deceased's race/ethnicity": "officer_race",
    "street address of incident": "incident_address",
    "city of incident": "incident_city",
    "county of incident": "incident_county",
    "zip code of incident": "incident_zip",
    "latitude of incident": "incident_latitude",
    "longitude of incident": "incident_longitude",
    "incident resulted in 1": "incident_result_1",
    "incident resulted in 2": "incident_result_2",
    "media": "media_link_1",
    "column1": "media_link_2",
    "column2": "media_link_3",
}

agency_col_renames = {
    "date of report %d": "agency_report_date_%d",
    "name of agency %d": "agency_name_%d",
    "city of agency %d": "agency_city_%d",
    "zip code of agency %d": "agency_zip_%d",
    "name of person %d filling out form": "agency_name_person_filling_out_%d",
    "email address of person %d filling out form": "agency_email_person_filling_out_%d",
}

for i in range(1, 3):
    for k, v in agency_col_renames.items():
        k = k % i
        v = v % i
        col_renames[k] = v


civilian_col_renames = {
    "non-peace officer's gender %d": "civilian_gender_%d",
    "non-peace officer's age %d": "civilian_age_%d",
    "non-peace officer's race/ethnicity %d": "civilian_race_%d",
    "non-officer first name %d": "civilian_name_first_%d",
    "non-officer last name %d": "civilian_name_last_%d",
}

for i in range(1, 4):
    for k, v in civilian_col_renames.items():
        k = k % i
        v = v % i
        col_renames[k] = v

In [10]:
shootings.columns = [col_renames[c] for c in shootings.columns]

### Translate columns about injury vs death to boolean

In [11]:
def death_injury_officer(s):
    if pd.isnull(s):
        return s
    elif 'death' in s.lower():
        return True
    elif 'injury' in s.lower():
        return False
    raise CleaningError('Invalid incident result: ' + s)

def death_injury_civilian(s):
    try:
        return death_injury_officer(s)
    except CleaningError as e:
        if 'suicide' in s.lower():
            return True
        raise

shootings['officer_died'] = shootings['incident_result_1'].apply(death_injury_officer)
shootings['civilian_died'] = shootings['incident_result_2'].apply(death_injury_civilian)
shootings['civilian_suicide'] = shootings['incident_result_2'].str.contains('suicide')
shootings.drop(['incident_result_1', 'incident_result_2'], axis=1, inplace=True)

### Replace 'Not Available' string values with true missing values

In [12]:
for c in shootings.columns:
    if shootings[c].dtype == 'object':
        shootings[c][shootings[c] == 'NOT AVAILABLE'] = None

/Users/everettwetchler/.virtualenvs/jupyter-data-processing/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### Convert "12345-6789" style zips into simply "12345"

In [13]:
for c in shootings.columns:
    if c.endswith('_zip'):
        shootings[c] = shootings[c].apply(lambda z: z if pd.isnull(z) else str(z).split('-')[0])

### Ensure ages are all valid integers

### Validate gender columns

In [14]:
validate_gender_cols(shootings)

### Make age columns numerical (float)

In [15]:
numericalize_age_cols(shootings)

### Simplify race names

In [16]:
standardize_race_cols(shootings)

### Convert date columns to ensure validity

In [17]:
for c in shootings.columns:
    if c.startswith('date_') or c.endswith('_date'):
        shootings[c] = pd.to_datetime(shootings[c])

In [18]:
shootings.head()

,date_ag_received,agency_report_date_1,agency_name_1,agency_city_1,agency_zip_1,date_incident,agency_name_person_filling_out_1,agency_email_person_filling_out_1,agency_report_date_2,agency_name_2,agency_city_2,agency_zip_2,agency_name_person_filling_out_2,agency_email_person_filling_out_2,officer_name_first,officer_name_last,officer_gender,officer_age,officer_race,incident_address,incident_city,incident_county,incident_zip,incident_latitude,incident_longitude,civilian_gender_1,civilian_age_1,civilian_race_1,civilian_name_first_1,civilian_name_last_1,civilian_gender_2,civilian_age_2,civilian_race_2,civilian_name_first_2,civilian_name_last_2,civilian_gender_3,civilian_age_3,civilian_race_3,civilian_name_first_3,civilian_name_last_3,media_link_1,media_link_2,media_link_3,officer_died,civilian_died,civilian_suicide
0,NaT,10/26/2015,SEGUIN POLICE DEPARTMENT,SEGUIN,78155,2015-10-06,MC MEYERS,MCMEYERS@SEGUINTEXAS.GOV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,54.0,WHITE,815 LAMAR DRIVE,SEGUIN,GUADALUPE,78155,29.576609,-97.947667,FEMALE,54.0,WHITE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN
1,NaT,11/3/2015,BEXAR COUNTY SHERIFF'S OFFICE,SAN ANTONIO,78207,2015-10-30,DET. FRANK STUBBS #4048,SSTUBBS@BEXAR.ORG,NaN,NaN,NaN,NaN,NaN,NaN,JOSEPH,CANALES,MALE,42.0,HISPANIC,8000 MIDCROWN,SAN ANTONIO,BEXAR,78218,29.507478,-98.380029,MALE,36.0,BLACK,SHERMAN,ROBINSON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MYSA,NEWS4SANANTONIO,NaN,False,NaN,NaN
2,2017-02-21,2/21/2017,MARLIN POLICE DEPARTMENT,MARLIN,76661,2015-11-01,DAMIEN EAGLIN,CHIEFEAGLIN@MARLINTX.NET,NaN,NaN,NaN,NaN,NaN,NaN,DARRELL,ALLEN,MALE,42.0,BLACK,301 W. AVENUE B,TEMPLE,BELL COUNTY,76501,31.095688,-97.344775,MALE,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WACO TRIBUNE-HERALD,NaN,NaN,True,NaN,NaN
3,NaT,12/29/2015,ODESSA POLICE DEPARTMENT,ODESSA,79761,2015-12-23,SGT. DAVID LARA,DLARA@ODESSA-TX.GOV,NaN,NaN,NaN,NaN,NaN,NaN,PETE,GONZALES,MALE,42.0,HISPANIC,1025 CUSTER AVE.,ODESSA,ECTOR,79761,31.865143,-102.335850,MALE,27.0,HISPANIC,ROY DANIEL,GARZA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CBS 7,BIG COUNTRY,NaN,False,NaN,NaN
4,NaT,12/29/2015,ODESSA POLICE DEPARTMENT,ODESSA,79761,2015-12-23,SGT. DAVID LARA,DLARA@ODESSA-TX.GOV,NaN,NaN,NaN,NaN,NaN,NaN,CORY,WESTER,MALE,38.0,WHITE,1025 CUSTER AVE.,ODESSA,ECTOR,79761,31.865143,-102.335850,MALE,27.0,HISPANIC,ROY DANIEL,GARZA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CBS 7,BIG COUNTRY,NaN,False,NaN,NaN


### Standardize agency names, and add agency county information

In [19]:
agency_to_county = dict(zip(agencies.agency, agencies.county))
for i in (1, 2):
    c = 'agency_name_%d' % i
    print("Standardizing", c)
    shootings[c] = shootings[c].apply(standardize_agency_name)
    shootings = insert_col_after(
        shootings,
        shootings[c].apply(lambda d: agency_to_county.get(d, np.nan)),
        'agency_county_%d' % i,
        'agency_city_%d' % i)

missing = shootings[shootings.agency_county_1.isnull()]
print('Missing county information for %d records' % len(missing))
missing.agency_name_1

Standardizing agency_name_1
Standardizing agency_name_2
Missing county information for 2 records


28    EL CENTRO COLLEGE POLICE DEPT
29    EL CENTRO COLLEGE POLICE DEPT
Name: agency_name_1, dtype: object

### Fix erroneous counties

In [20]:
counties = set(agencies.county)
shootings[~shootings.incident_county.isin(counties)]

,date_ag_received,agency_report_date_1,agency_name_1,agency_city_1,agency_county_1,agency_zip_1,date_incident,agency_name_person_filling_out_1,agency_email_person_filling_out_1,agency_report_date_2,agency_name_2,agency_city_2,agency_county_2,agency_zip_2,agency_name_person_filling_out_2,agency_email_person_filling_out_2,officer_name_first,officer_name_last,officer_gender,officer_age,officer_race,incident_address,incident_city,incident_county,incident_zip,incident_latitude,incident_longitude,civilian_gender_1,civilian_age_1,civilian_race_1,civilian_name_first_1,civilian_name_last_1,civilian_gender_2,civilian_age_2,civilian_race_2,civilian_name_first_2,civilian_name_last_2,civilian_gender_3,civilian_age_3,civilian_race_3,civilian_name_first_3,civilian_name_last_3,media_link_1,media_link_2,media_link_3,officer_died,civilian_died,civilian_suicide
2,2017-02-21,2/21/2017,MARLIN POLICE DEPT,MARLIN,FALLS,76661,2015-11-01,DAMIEN EAGLIN,CHIEFEAGLIN@MARLINTX.NET,NaN,None,NaN,NaN,NaN,NaN,NaN,DARRELL,ALLEN,MALE,42.0,BLACK,301 W. AVENUE B,TEMPLE,BELL COUNTY,76501,31.095688,-97.344775,MALE,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WACO TRIBUNE-HERALD,NaN,NaN,True,NaN,NaN
50,2017-05-17,5/17/2017,KILLEEN POLICE DEPT,KILLEEN,BELL,76542,2017-04-24,CDR. ALEX GEARHART,AGEARHART@KILLEENTEXAS.GOV,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MALE,40.0,WHITE,3207 E. CENTRAL TEXAS EXPRESSWAY,KILLEEN,BELL COUNTY,76543,31.087794,-97.707175,MALE,15.0,BLACK,DAEZION CHRISTIAN PAUL,TURNER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KDH NEWS,NaN,NaN,False,True,False
71,2018-02-23,2/22/2018,RICHARDSON POLICE DEPT,RICHARDSON,DALLAS,75081,2018-02-07,SGT. KEVIN W. PERLICH,KEVIN.PERLICH@COR.GOV,NaN,None,NaN,NaN,NaN,NaN,NaN,DAVID STEFAN,SHERRARD,MALE,37.0,WHITE,"3500 NORTH STAR ROAD, BLDG. 12, APT. 1235",RICHARDSON,COLIN,75082,32.999521,-96.640781,MALE,26.0,WHITE,BRANDON,MCCALL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DALLAS MORNING NEWS,NBC DFW,NaN,True,NaN,NaN


In [21]:
fixes = {
    'COLIN': 'COLLIN',
    'BELL COUNTY': 'BELL'
}
for before, after in fixes.items():
    shootings.loc[shootings.incident_county == before, 'incident_county'] = after

shootings[~shootings.incident_county.isin(counties)]

,date_ag_received,agency_report_date_1,agency_name_1,agency_city_1,agency_county_1,agency_zip_1,date_incident,agency_name_person_filling_out_1,agency_email_person_filling_out_1,agency_report_date_2,agency_name_2,agency_city_2,agency_county_2,agency_zip_2,agency_name_person_filling_out_2,agency_email_person_filling_out_2,officer_name_first,officer_name_last,officer_gender,officer_age,officer_race,incident_address,incident_city,incident_county,incident_zip,incident_latitude,incident_longitude,civilian_gender_1,civilian_age_1,civilian_race_1,civilian_name_first_1,civilian_name_last_1,civilian_gender_2,civilian_age_2,civilian_race_2,civilian_name_first_2,civilian_name_last_2,civilian_gender_3,civilian_age_3,civilian_race_3,civilian_name_first_3,civilian_name_last_3,media_link_1,media_link_2,media_link_3,officer_died,civilian_died,civilian_suicide


### Re-order columns more sensibly
* **Disclaimer**: This code is gross and probably an OCD attempt to make the final result look nice. Just destroy this cell if it proves too confusing or opaque.

In [22]:
section_order = ['date', 'incident', 'officer', 'agency', 'civilian', 'media']
civilian_col_order = ['name_first', 'name_last', 'age', 'race', 'gender']
officer_col_order = ['died', 'name_first', 'name_last', 'age', 'race', 'gender']
agency_col_order = ['name', 'city', 'county', 'zip', 'report_date', 'name_person_filling_out', 'email_person_filling_out']
new_col_order = []
remaining = set(shootings.columns)
for s in section_order:
    if s == 'civilian':
        for c in ['civilian_died', 'civilian_suicide']:
            new_col_order.append(c)
            remaining.remove(c)
        for i in range(1, 4):
            for field in civilian_col_order:
                c = 'civilian_%s_%d' % (field, i)
                new_col_order.append(c)
                remaining.remove(c)
    elif s == 'officer':
        for field in officer_col_order:
            c = 'officer_%s' % field
            new_col_order.append(c)
            remaining.remove(c)
    elif s == 'agency':
        for i in range(1, 3):
            for field in agency_col_order:
                c = 'agency_%s_%d' % (field, i)
                new_col_order.append(c)
                remaining.remove(c)
    else:
        this_section_cols = []
        for c in shootings.columns:
            if c.startswith(s + '_'):
                this_section_cols.append(c)
                remaining.remove(c)
        new_col_order.extend(sorted(this_section_cols))

assert(len(remaining) == 0)
shape_before = shootings.shape
shootings = shootings[new_col_order]
assert(shootings.shape == shape_before)

In [23]:
shootings.sort_values(['date_incident', 'incident_county', 'agency_county_1'], inplace=True)
shootings.head()

,date_ag_received,date_incident,incident_address,incident_city,incident_county,incident_latitude,incident_longitude,incident_zip,officer_died,officer_name_first,officer_name_last,officer_age,officer_race,officer_gender,agency_name_1,agency_city_1,agency_county_1,agency_zip_1,agency_report_date_1,agency_name_person_filling_out_1,agency_email_person_filling_out_1,agency_name_2,agency_city_2,agency_county_2,agency_zip_2,agency_report_date_2,agency_name_person_filling_out_2,agency_email_person_filling_out_2,civilian_died,civilian_suicide,civilian_name_first_1,civilian_name_last_1,civilian_age_1,civilian_race_1,civilian_gender_1,civilian_name_first_2,civilian_name_last_2,civilian_age_2,civilian_race_2,civilian_gender_2,civilian_name_first_3,civilian_name_last_3,civilian_age_3,civilian_race_3,civilian_gender_3,media_link_1,media_link_2,media_link_3
0,NaT,2015-10-06,815 LAMAR DRIVE,SEGUIN,GUADALUPE,29.576609,-97.947667,78155,False,NaN,NaN,54.0,WHITE,FEMALE,SEGUIN POLICE DEPT,SEGUIN,GUADALUPE,78155,10/26/2015,MC MEYERS,MCMEYERS@SEGUINTEXAS.GOV,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.0,WHITE,FEMALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaT,2015-10-30,8000 MIDCROWN,SAN ANTONIO,BEXAR,29.507478,-98.380029,78218,False,JOSEPH,CANALES,42.0,HISPANIC,MALE,BEXAR CO SHERIFFS OFFICE,SAN ANTONIO,BEXAR,78207,11/3/2015,DET. FRANK STUBBS #4048,SSTUBBS@BEXAR.ORG,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SHERMAN,ROBINSON,36.0,BLACK,MALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MYSA,NEWS4SANANTONIO,NaN
2,2017-02-21,2015-11-01,301 W. AVENUE B,TEMPLE,BELL,31.095688,-97.344775,76501,True,DARRELL,ALLEN,42.0,BLACK,MALE,MARLIN POLICE DEPT,MARLIN,FALLS,76661,2/21/2017,DAMIEN EAGLIN,CHIEFEAGLIN@MARLINTX.NET,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,MALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WACO TRIBUNE-HERALD,NaN,NaN
3,NaT,2015-12-23,1025 CUSTER AVE.,ODESSA,ECTOR,31.865143,-102.335850,79761,False,PETE,GONZALES,42.0,HISPANIC,MALE,ODESSA POLICE DEPT,ODESSA,ECTOR,79761,12/29/2015,SGT. DAVID LARA,DLARA@ODESSA-TX.GOV,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ROY DANIEL,GARZA,27.0,HISPANIC,MALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CBS 7,BIG COUNTRY,NaN
4,NaT,2015-12-23,1025 CUSTER AVE.,ODESSA,ECTOR,31.865143,-102.335850,79761,False,CORY,WESTER,38.0,WHITE,MALE,ODESSA POLICE DEPT,ODESSA,ECTOR,79761,12/29/2015,SGT. DAVID LARA,DLARA@ODESSA-TX.GOV,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ROY DANIEL,GARZA,27.0,HISPANIC,MALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CBS 7,BIG COUNTRY,NaN


## 3. Write

In [24]:
shootings.to_csv(CLEANED_FILENAME, index=False)